In [1]:
import os
import gc
import time
from tqdm import tqdm
import itertools
from subprocess import call
from collections import deque
import pickle 
import numpy as np
import pandas as pd
import networkx as nx
from numba import jit
from multiprocessing import Pool
from functools import partial
import scipy
from scipy import linalg, sparse
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import average_precision_score, f1_score, roc_auc_score

In [2]:
adj_matrix = sparse.load_npz('./data/preprocessed/blogcatalog/adj_matrix.npz')
adj_dict = pickle.load(open('./data/preprocessed/blogcatalog/adj_dict.pkl', 'rb'))
node_labels = pickle.load(open('./data/preprocessed/blogcatalog/node_labels.pkl', 'rb'))

In [3]:
def spectral_clustering(adj_matrix, dim=128):
    n, m = adj_matrix.shape
    diags = adj_matrix.sum(axis=1).flatten()
    D = sparse.spdiags(diags, [0], m, n, format='csr')
    L = D - adj_matrix
    with scipy.errstate(divide='ignore'):
        diags_sqrt = 1.0 / scipy.sqrt(diags)
    diags_sqrt[scipy.isinf(diags_sqrt)] = 0
    DH = sparse.spdiags(diags_sqrt, [0], m, n, format='csr')
    laplacian = DH.dot(L.dot(DH))

    _, v = sparse.linalg.eigs(laplacian, k=dim + 1, which='SM')
    embedding = v[:, 1:].real
    return embedding

In [48]:
# embedding = spectral_clustering(adj_matrix.astype(np.int16), dim=128)
scaler = StandardScaler()
embedding = scaler.fit_transform(embedding)

In [19]:
embedding_2 = spectral_clustering(adj_matrix, dim=128)
scaler = StandardScaler()
embedding_2 = scaler.fit_transform(embedding_2)

## Node Classification

In [49]:
X_train, X_test, y_train, y_test = train_test_split(embedding, node_labels, test_size=0.2, random_state=21)

In [50]:
clf = OneVsRestClassifier(LogisticRegression(), n_jobs=12)
clf.fit(X_train, y_train)

OneVsRestClassifier(estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
          n_jobs=12)

In [51]:
preds = clf.predict(X_test)

In [52]:
f1_score(y_test, preds, average='weighted')

/home/ubuntu/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


0.27166209858445056

In [40]:
def node_classification_kfold(embedding, node_labels, n_splits=5, random_state=21):
    kf = KFold(n_splits=n_splits, random_state=random_state, shuffle=True)
    score_f1_kfold = []
    score_ap_kfold = []
    for i, (index_train, index_test) in tqdm(enumerate(kf.split(node_labels))):
        X_train, X_test = embedding[index_train], embedding[index_test]
        y_train, y_test = node_labels[index_train], node_labels[index_test]
        
        clf = OneVsRestClassifier(LogisticRegression(), n_jobs=21)
        clf.fit(X_train, y_train)
        
        preds_label = clf.predict(X_test)
        preds_proba = clf.predict_proba(X_test)
        
        score_f1 = f1_score(y_test, preds_label, average='weighted')
        score_ap = average_precision_score(y_test, preds_proba, average='weighted')
        score_f1_kfold.append(score_f1)
        score_ap_kfold.append(score_ap)
        
    return score_f1_kfold, score_ap_kfold

In [56]:
node_labels

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [53]:
score_f1, score_ap = node_classification_kfold(embeddings, node_labels)

0it [00:00, ?it/s]/home/ubuntu/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
5it [00:09,  1.81s/it]


In [54]:
np.mean(score_f1)

0.266720395265443

In [55]:
np.mean(score_ap)

0.35253627881128524

In [44]:
np.std(score_f1)

0.010745377351144968

In [45]:
np.std(score_ap)

0.01187607619124917

## Link Prediction

In [10]:
@jit('i8[:](i8[:, :])')
def get_edge_list(adj_matrix):
    edge_list = []
    start_nodes = np.arange(adj_matrix.shape[0])
    for start in start_nodes:
        end_nodes = np.nonzero(adj_matrix[start].toarray().reshape(-1, ))[0]
        end_nodes = end_nodes[end_nodes > start]
        for end in end_nodes:
            edge_list.append((start, end))
    
    return np.array(edge_list)

In [11]:
edge_list = get_edge_list(adj_matrix)
nodes = list(range(adj_matrix.shape[0]))

In [12]:
@jit('i8[:, :](i8[:, :], f8)')
def sample_negative_links(edge_list, neg_ratio=1.0):
    nodes = np.unique(edge_list)
    index_shuffle = np.random.permutation(np.arange(len(nodes)))
    nodes = nodes[index_shuffle]
    
    neg_size = int(len(edge_list) * neg_ratio)
    result = deque([])
    for i, start in enumerate(nodes):
        end_nodes = nodes[nodes > start]
        drop_nodes = edge_list[edge_list[:, 0] == start][:, 1]
        end_nodes = np.setdiff1d(end_nodes, drop_nodes)
        for end in end_nodes:
            edge = np.array([start, end], dtype=int)
            result.append(edge)
            if len(result) == neg_size:
                break
        if len(result) == neg_size:
                break
    return np.array(result)


In [13]:
@jit('Tuple((i8[:, :], i8[:, :]))(i8[:, :], f8)')
def split_graph(edge_list, test_ratio=0.2):
    node_degree = np.bincount(np.sort(edge_list.ravel()))
    nodes = np.unique(edge_list)
    split_train = deque([])
    split_test = deque([])
    for n in nodes:
        edges = edge_list[edge_list[:, 0] == n]
        num_split_test = int(node_degree[n] * test_ratio)
        num_split_train = int(node_degree[n] * (1 - test_ratio))
        if num_split_train <= 1:
            split_test.append(edges)
            split_train.append(edges)
            continue
            
        candidates = deque([])
        for edge in edges:
            if node_degree[edge[1]] > 1:
                candidates.append(edge)
            else:
                split_test.append(edge.reshape(1, 2))
                split_train.append(edge.reshape(1, 2))
        candidates = np.array(candidates)
        if len(candidates) < 1:
            continue
        split_test_ends = np.random.choice(candidates[:, 1], num_split_test)
        split_train_ends = np.setdiff1d(edges[:, 1], split_test_ends)
        
        split_test_edges = np.ones([len(split_test_ends), 2], dtype=int) * n
        split_train_edges = np.ones([len(split_train_ends), 2], dtype=int) * n
        
        split_test_edges[:, 1] = split_test_ends
        split_train_edges[:, 1] = split_train_ends
        split_test.append(split_test_edges)
        split_train.append(split_train_edges)
        
        for end in split_test_ends:
            node_degree[end] -= 1
    
    return np.concatenate(split_train, axis=0), np.concatenate(split_test, axis=0)

In [32]:
edge_list_train, edge_list_test = split_graph(edge_list, 0.2)

In [33]:
edge_list_train.shape

(244847, 2)

In [34]:
edge_list_test.shape

(108709, 2)

In [35]:
edge_list_train.shape[0] + edge_list_test.shape[0] - 13174

340382

In [36]:
edge_list.shape

(333983, 2)

In [14]:
since = time.time()
negative_links = sample_negative_links(edge_list_train, neg_ratio=1)
print(time.time() - since)

1.2557430267333984


In [77]:
since = time.time()
embedding = spectral_clustering(adj_matrix, dim=128)
print(time.time() - since)

4.839042901992798


In [84]:
since = time.time()
embedding = spectral_clustering(adj_matrix_train, dim=128)
print(time.time() - since)

4.931436538696289


In [14]:
@jit
def edgelist_to_matrix(edge_list):
    n = len(np.unique(edge_list))
    adj_matrix = np.zeros([n, n], dtype=np.int16)
    for edge in edge_list:
        adj_matrix[edge[0], edge[1]] = 1
        adj_matrix[edge[1], edge[0]] = 1
    return sparse.csr_matrix(adj_matrix)

In [83]:
adj_matrix_train = edgelist_to_matrix(edge_list_train)

In [90]:
negative_links = sample_negative_links(edge_list_train, 1.0)
edges_train = np.concatenate([edge_list_train, negative_links], axis=0)
labels_train = np.concatenate([np.ones(len(edge_list_train), dtype=int), np.zeros(len(negative_links), dtype=int)])
index_shuffle = np.random.permutation(np.arange(len(edges_train)))
edges_train, labels_train = edges_train[index_shuffle], labels_train[index_shuffle]

In [92]:
X_train = embedding[edges_train[:, 0], :] * embedding[edges_train[:, 1], :]
X_train.shape

(489694, 128)

In [15]:
def link_prediction(edge_list, mode='Hadamard', test_ratio=0.2, neg_ratio=1.0, n_trials=5, random_state=21):
    np.random.seed(random_state)
    scores_f1 = []
    scores_ap = []
    scores_auc = []
    for i in range(n_trials):
        print(f'Trial {i+1:d} / {n_trials:d}')
        print('Split the graph into train and test.')
        edge_list_train, edge_list_test = split_graph(edge_list, test_ratio)
        adj_matrix_train = edgelist_to_matrix(edge_list_train)
        
        print('Compute node embeddings on the train graph split.')
#         embedding = spectral_clustering(adj_matrix_train, dim=128)
        model = Node2Vec(dim=128, walk_length=80, num_walks=10, context_size=10, epochs=1, return_param=1, inout_param=1)
        embedding = model.learn_embedding(adj_matrix_train)
        
        print('Sample negative links for the train graph split.')
        negative_links = sample_negative_links(edge_list_train, neg_ratio)
        edges_train = np.concatenate([edge_list_train, negative_links], axis=0)
        labels_train = np.concatenate([np.ones(len(edge_list_train), dtype=int), np.zeros(len(negative_links), dtype=int)])
        index_shuffle = np.random.permutation(np.arange(len(edges_train)))
        edges_train, labels_train = edges_train[index_shuffle], labels_train[index_shuffle]
        
        print('Sample negative links for the test graph split.')
        negative_links = sample_negative_links(edge_list_test, neg_ratio)
        edges_test = np.concatenate([edge_list_test, negative_links], axis=0)
        labels_test = np.concatenate([np.ones(len(edge_list_test), dtype=int), np.zeros(len(negative_links), dtype=int)])
        
        if mode == 'Hadamard':
            X_train = embedding[edges_train[:, 0], :] * embedding[edges_train[:, 1], :]
            X_test = embedding[edges_test[:, 0], :] * embedding[edges_test[:, 1], :]
        else:
            X_train = embedding[edges_train[:, 0], :] + embedding[edges_train[:, 1], :]
            X_test = embedding[edges_test[:, 0], :] + embedding[edges_test[:, 1], :]
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)
        y_train, y_test = labels_train, labels_test
        
        print('Train a classifier.')
        clf = LogisticRegression(solver='saga')
        clf.fit(X_train, y_train)
        
        print('Make link predictions.')
        preds_label = clf.predict(X_test)
        preds_proba = clf.predict_proba(X_test)[:, 1]
        
        score_f1 = f1_score(y_test, preds_label)
        score_ap = average_precision_score(y_test, preds_proba)
        score_auc = roc_auc_score(y_test, preds_proba)
        
        scores_f1.append(score_f1)
        scores_ap.append(score_ap)
        scores_auc.append(score_auc)
        
        print()
        
    return scores_f1, scores_ap, scores_auc
    

In [16]:
scores_f1, scores_ap, scores_auc = link_prediction(edge_list)

Trial 1 / 5
Split the graph into train and test.


KeyboardInterrupt: 

In [56]:
np.mean(scores_ap)

0.9071859961293571

In [53]:
np.std(scores_ap)

0.005836689585639017

In [22]:
@jit('i8[:](i8[:, :])')
def matrix_to_edgelist(adj_matrix):
    edge_list = []
    start_nodes = np.arange(adj_matrix.shape[0])
    for start in start_nodes:
        end_nodes = np.nonzero(adj_matrix[start].toarray().reshape(-1, ))[0]
        end_nodes = end_nodes[end_nodes > start]
        for end in end_nodes:
            edge_list.append((start, end))

    return np.array(edge_list)

In [23]:
class Node2Vec:
    def __init__(self, dim, walk_length=80, num_walks=10, context_size=10, epochs=1,
                 return_param=1, inout_param=1, directed=False, weighted=False, verbose=True, bin_path='node2vec',
                 graph_path='./data/tmp/tmp.graph', embed_path='./data/embeddings/node2vec.emb'):

        self.dim = dim
        self.walk_length = walk_length
        self.num_walks = num_walks
        self.context_size = context_size
        self.epochs = epochs
        self.return_param = return_param
        self.inout_param = inout_param
        self.directed = directed
        self.weighted = weighted
        self.verbose = verbose

        self.bin_path = bin_path
        self.graph_path = graph_path
        self.embed_path = embed_path

    def learn_embedding(self, adj_matrix):
        args = [os.path.expanduser(self.bin_path)]

        edge_list = matrix_to_edgelist(adj_matrix)
        self.save_edgelist(edge_list, self.graph_path)

        args.append(f"-i:{self.graph_path}")
        args.append(f"-o:{self.embed_path}")
        args.append(f"-d:{self.dim:d}")
        args.append(f"-l:{self.walk_length:d}")
        args.append(f"-r:{self.num_walks:d}")
        args.append(f"-k:{self.context_size:d}")
        args.append(f"-e:{self.epochs:d}")
        args.append(f"-p:{self.return_param:.6f}")
        args.append(f"-q:{self.inout_param:.6f}")
        if self.directed:
            args.append("-dr")
        if self.weighted:
            args.append("-w")
        if self.verbose:
            args.append("-v")

        try:
            call(args)
        except Exception as e:
            print(str(e))
            raise Exception('node2vec not found. Please compile snap, place node2vec in the path')
        embeddings = self.load_embeddings()
        return embeddings

    def load_embeddings(self):
        with open(self.embed_path, 'r') as f:
            n, d = f.readline().strip().split()
            X = np.zeros([int(n), int(d)], dtype=np.float32)
            for line in f:
                emb = line.strip().split()
                emb_fl = [float(emb_i) for emb_i in emb[1:]]
                X[int(emb[0]), :] = emb_fl
        return X

    @staticmethod
    def save_edgelist(edge_list, path):
        lines = ''
        for edge in edge_list:
            lines += str(edge[0]) + ' ' + str(edge[1]) + '\n'
        with open(path, 'w') as f:
            f.write(lines)

In [24]:
model = Node2Vec(dim=128, walk_length=80, num_walks=10, context_size=10, epochs=3, return_param=1, inout_param=1)

In [25]:
embeddings = model.learn_embedding(adj_matrix)

In [46]:
embedding = embeddings

In [27]:
i = 0
with open('./data/embeddings/node2vec.emb', 'r') as f:
    n, d = f.readline().strip().split()
#     X = np.zeros([int(n), int(d)], dtype=np.float32)
    for line in f:
        i += 1
        emb = line.strip().split()
        emb_fl = [float(emb_i) for emb_i in emb[1:]]
#         X[int(emb[0]), :] = emb_fl

In [47]:
embeddings

array([[ 0.0807006 ,  0.0971116 ,  0.0733267 , ..., -0.362133  ,
         0.177519  ,  0.0287865 ],
       [-0.150951  , -0.0541495 ,  0.0384471 , ..., -0.382873  ,
         0.399381  , -0.219226  ],
       [-0.0297509 , -0.119281  , -0.0182671 , ..., -0.204567  ,
         0.0411328 , -0.109958  ],
       ...,
       [-0.010897  ,  0.0793153 ,  0.00592847, ...,  0.443282  ,
        -0.0537316 , -0.0853873 ],
       [-0.0492875 , -0.226037  , -0.229748  , ...,  0.256811  ,
         0.237568  , -0.214519  ],
       [ 0.117696  , -0.0646967 , -0.221989  , ...,  0.125293  ,
         0.283742  , -0.10564   ]], dtype=float32)